In [42]:
using LinearAlgebra 
using Plots #Graph
using SparseArrays #Sparse matrices 
using LaTeXStrings #Titles and labels 
using JLD #Save data 
#definition of N,M,D
global N=7
global M=7
global D=Int((factorial(M+N-1))/(factorial(M-1)*factorial(N)))  

#i-prime number function(suggested form) 
p(i)=100i+3 

#Generate a basis for N = M
function generate_basis(N, M)
    D = prod(max(N, M):N+M-1) ÷ prod(1:min(N, M))
    basis = [zeros(Int, M) for _ in 1:D]
    basis[1][1] = N
    for t = 2:D
        if basis[t-1][M] != 0
            k = M - 1
        else
            k = M
        end
        while k > 0 && basis[t-1][k] == 0
            k -= 1
        end
        @views basis[t][1:k-1] .= basis[t-1][1:k-1]
        basis[t][k] = basis[t-1][k] - 1
        basis[t][k+1] = N - sum(@view(basis[t][1:k])) 
        end 
    return basis  
end

generate_basis (generic function with 1 method)

In [43]:
v=generate_basis(N,M);

In [44]:
function occupation(i, v) 
     if(v[i] >= 1)  
        return  v[i]  
    else 
        return 0;
    end  
end 

occupation (generic function with 1 method)

In [45]:
function matrixoccupation(k, D) 
    mat = spzeros(Complex{Float64}, D,D)
    for j in 1:D 
        mat[j,j] = occupation(k, v[j]) 
    end   
    return mat 
end

matrixoccupation (generic function with 1 method)

In [46]:
#sum( matrixoccupation(i,D) for i in 2:(N-1) ) == matrixoccupation(2,D)

In [52]:
#Parity order parameter 
theta = pi
Op = exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N-1))) - (N-1)*I) 
Op = real(Op);

In [48]:
#Sum until N
theta = pi
Op = exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N))) - (N)*I) 
Op = real(Op);

In [53]:
#String Order parameter (we fix the parameters i =1, j = N, and i<=k<j )
Os = (Matrix(matrixoccupation(1,D)) - I)*(exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N-1))) - (N-1)*I))*(Matrix(matrixoccupation(N,D)) - I) 
Os = real(Os);

In [50]:
#Sum until N 
Os = (Matrix(matrixoccupation(1,D)) - I)*(exp(1im*theta*Matrix(sum(matrixoccupation(i,D) for i in 1:(N))) - (N)*I))*(Matrix(matrixoccupation(N,D)) - I) 
Os = real(Os);

In [37]:
#Optim Example for 
#f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2  
#x0 = [0.0, 0.0]  
#optimize(f, x0)  
# optimize(f, zeros(2) )  
##optimize(x->dot(x, Os*x), zeros(D)) ###NO CONSTRAINTS! so we will use JuMP package 
using JuMP
import Ipopt

In [21]:
function ParityOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Min, z)  
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Op = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

#ParityOrder()

ParityOrder (generic function with 1 method)

In [22]:
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, 0 <= x[1:D] <= 1 )
    @objective(model, Max, z)   
    @constraint(model, dot(x,Op*x) == 0)
    @constraint(model, dot(x, Os*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Os = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

#StringOrder() 
#println("square root of the probability amplitude ")

StringOrder (generic function with 1 method)

In [51]:
#Until N
coefficients = Vector{Float64}()
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, 0 <= x[1:D] <= 1 )
    @objective(model, Max, z)   
    @constraint(model, sum(Op*x) == 0)
    @constraint(model, sum(Os*x) == z)
    @constraint(model, sum(x) == 1)
    optimize!(model)
    if verbose
        #print(model)
        #println("Objective value: ", objective_value(model)) 
        #L"Objective~value:"
        println("Os = ", value(z))
        for i in 1:D 
            #println("x_$i = ", value(x[i]) )  
            push!(coefficients, value(x[i]))
        end
    end
    return
end

StringOrder()
DictOfBasis = Dict(coefficients .=> v) 
println("The largest probability amplitude is ", maximum(coefficients) ," corresponding to the basis vector " , 
DictOfBasis[maximum(coefficients)] )

Os = 0.0013038442708021819
The largest probability amplitude is 0.10291073175667771 corresponding to the basis vector [7, 0, 0, 0, 0, 0, 0]


In [54]:
#Until N-1 
coefficients = Vector{Float64}()
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, 0 <= x[1:D] <= 1 )
    @objective(model, Max, z)   
    @constraint(model, sum(Op*x) == 0)
    @constraint(model, sum(Os*x) == z)
    @constraint(model, sum(x) == 1)
    optimize!(model)
    if verbose
        #print(model)
        #println("Objective value: ", objective_value(model)) 
        #L"Objective~value:"
        println("Os = ", value(z))
        for i in 1:D 
            #println("x_$i = ", value(x[i]) )  
            push!(coefficients, value(x[i]))
        end
    end
    return
end

StringOrder()
DictOfBasis = Dict(coefficients .=> v) 
println("The largest probability amplitude is ", maximum(coefficients) ," corresponding to the basis vector " , 
DictOfBasis[maximum(coefficients)] )

Os = 0.014868470005176038
The largest probability amplitude is 0.4999042362124672 corresponding to the basis vector [4, 0, 0, 0, 0, 0, 3]
